# Energy Materials: Design, Discovery and Data

## 3. Design Principles for Energy Technologies

![](./images/design.png)
<center>From [Nature Chemistry](http://www.nature.com/nchem/journal/v7/n4/full/nchem.2213.html) (2015)</center>

## Advance Reading
Read [TE Design Lab: A virtual laboratory for thermoelectric material design](http://www.sciencedirect.com/science/article/pii/S0927025615007132) (Compuational Materials Science, 2016)

See also: http://www.tedesignlab.org

## Lecture Slides 
On [Speakerdeck](https://speakerdeck.com/aronwalsh)

In [1]:
%%HTML 
<script async class="speakerdeck-embed" data-id="f4e158ee57bf433fb6eff222ae083cdc" data-ratio="1.33333333333333" src="//speakerdeck.com/assets/embed.js"></script>

## pyExample: Effective Mass Theory 

Improved from https://github.com/WMD-group/effectivemasstheory 

Effective mass theory is a useful tool in semiconductor physics for gaining insights into the properties of electrons and holes, including exciton binding energies, quantum confinement in nanostructures, and defect transitions. The concept is that carriers in semiconductors behave like free electrons except with a modified mass that is determined by the band structure. This is not always true, but it is good place to start from. The theory is covered well in [Yu and Cardona's textbook](https://books.google.co.uk/books/about/Fundamentals_of_Semiconductors.html?id=W9pdJZoAeyEC&redir_esc=y).

### Python Setup

In [26]:
import sys
assert sys.version_info >= (3,4)
import math as m
import scipy.constants as sc
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Input: Materials Properties

We require a set of properties that define the bulk electronic and dielectric properties of our material (effective masses, dielectric constants and an optical phonon frequency)

In [27]:
## Default is CH3NH3PbI3
h = 0.12 # hole effective mass / m_e
e = 0.15 # electron effective mass / m_e
d0 = 33 # static dielectric constant / epsilon
d1 = 4.5 # optical dielectric constant / epsilon 
lo = 2.25 # optical phonon frequency / THz

# Reduced mass and screening
mass=((e*h)/(e+h))
diel=(1/d1-1/d0)

print ("Effective screening: " + str(1/diel) + " epsilon")
print ("Reduced mass: " + str(mass) + " m_e")

Effective screening: 5.210526315789474 epsilon
Reduced mass: 0.06666666666666665 m_e


Electrons and hole carriers can interact to form a new quasi-particle, an exciton, with a characteristic radius and binding energy

### Calculate: Exciton Properties

In [31]:
radius_bohr=(d0/mass)
radius=(d0/mass)*0.529177249 # Bohr to Angstrom
radius_h=(d0/h)*0.529177249 # Bohr to Angstrom
radius_e=(d0/e)*0.529177249 # Bohr to Angstrom

print ("Shallow acceptor radius: %3.2f A (%3.2f nm)" %(radius_h, radius_h/10))
print ("Shallow donor radius: %3.2f A (%3.2f nm) \n" %(radius_e, radius_e/10))    

# Binding energy from Mott-Wannier theory
binding=((1/(d0*radius_bohr))*(13.605698066*1000))

print ("Exciton radius: %3.2f A" %(radius))
print ("Exciton binding energy: %3.2f meV" %(binding))    

Shallow acceptor radius: 145.52 A (14.55 nm)
Shallow donor radius: 116.42 A (11.64 nm) 

Exciton radius: 261.94 A
Exciton binding energy: 0.83 meV


#### Task: Modify the above code to calculate and print the instanteous and thermalised exciton binding energy 

### Calculate: Polarons

The analysis above is based on the effective mass of an electron or hole from the curvature of the electronic band structure - sometimes termed a Bloch effective mass. In reality, an electron or hole will interact with the lattice and form a new quasi-particle, a polaron. 

In [32]:
## From Mott (1968)
radius_bohr_h=(2/(h*diel))
radius_bohr_e=(2/(e*diel))
radius_bh=(2/(h*diel))*0.529177249
radius_be=(2/(e*diel))*0.529177249

print ("Hole band polaron radius: %3.2f A" %(radius_bh))
print ("Electron band polaron radius: %3.2f A" %(radius_be))

# Include lattice vibrations - phonons
freq = lo*0.0003039659692 # THz to Ry
h_alpha=(1/2)*diel*m.sqrt(2*h/freq)
e_alpha=(1/2)*diel*m.sqrt(2*e/freq)

print ("\nPolaron coupling constants: %3.2f  %3.2f" %(h_alpha, e_alpha))

# Small polaron mass (Feynman)
h_pol=h*(1+h_alpha/6)
e_pol=e*(1+e_alpha/6)

print ("Renormalised masses: %3.2f m_e (+%3.0f%%)  %3.2f m_e (+%3.0f%%)" %(h_pol,(100*h_alpha/6),e_pol,(100*e_alpha/6)))

# Small polaron radius
radius_bhp=(2/(h_pol*diel))*0.529177249
radius_bep=(2/(e_pol*diel))*0.529177249

print ("\nHole large polaron radius: %3.2f A" %(radius_bhp))
print ("Electron large polaron radius: %3.2f A" %(radius_bep))

Hole band polaron radius: 45.95 A
Electron band polaron radius: 36.76 A

Polaron coupling constants: 1.80  2.01
Renormalised masses: 0.16 m_e (+ 30%)  0.20 m_e (+ 33%)

Hole large polaron radius: 35.36 A
Electron large polaron radius: 27.54 A


#### Task: If the lattice constant of the cubic unit cell is 6.3 A, how many unit cells is an electronic carrier spread over? 

### Calculate: Mott Semiconductor-Metal Transition

When the concentrations of electrons or holes in a semiconductors is high enough so that their wavefunctions overlap, there is an electronic transition where the metal starts to repond like a metal. The critical concentration is determined by the Mott criteria. The original theory was by [Mott](http://www.tandfonline.com/doi/abs/10.1080/14786436108243318) and extended to a large family of materials by [Edwards and Seinko (1978)](http://journals.aps.org/prb/abstract/10.1103/PhysRevB.17.2575).

In [33]:
# Exciton transition ~ 1/exciton volume (Optical properties of Solids - Mark Fox)
mott=((1/(4/3*sc.pi*(radius_bohr**3)))*(188971616.463**3))
print ("Exciton transition: %3.0e cm-3" %(mott))

# Mott transition (holes)
mott=(((0.26/radius_bohr_h)**3)*(188971616.463**3))
print ("Hole transition: p = %3.0e cm-3" %(mott))
    
# Mott transition (electrons)
mott=(((0.26/radius_bohr_e)**3)*(188971616.463**3))
print ("Electron transition: n = %3.0e cm-3" %(mott))
    
# Note that the value of 0.26 for the Mott Criteron is taken from:
# "Universality aspects of the metal-nonmetal transition in condensed media"
# Edwards and Seinko, PRB 17, 2575 (1978) 

Exciton transition: 1e+16 cm-3
Hole transition: p = 2e+17 cm-3
Electron transition: n = 4e+17 cm-3


## Supplementary Links 

* Thermoelectrics: http://www.tedesignlab.org
* Batteries: https://materialsproject.org/#search/batteries/
* Superalloys: http://aflowlib.org/superalloys/

